# 55 Public, Cleveland Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each scenario separately. These files will be used in the Jupyter Notebook.

3. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion

4. Rerun batch sims in EnergyPlus v9.5.
5. Add scenario names, elevations, and glazing systems to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
resgas_cost = 1.16
reselec_cost = 0.1238

comgas_cost = 0.56
comelec_cost = 0.0972

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.93

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MWh) 1,436.6 (lb/kWh) 1.4366
####Electricity (lb/MMBtu) 421.03 (lb/kbtu) .42103 ####
####Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####
#Renewable Natural Gas (Building Used, lb/MMBtu) 50.22
#No.2 Fuel Oil (lb/MMBtu) 195.64
#Propane (lb/MMBtu) 168.50
#Renewable Propane (lb/MMBtu) 46.89
#Natural Gas (mCHP NG Engine Used, lb/MMBtu) 176.24
#Natural Gas (mCHP Fuel Cell Used, lb/MMBtu) 142.30

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI

sf = 329.55

In [6]:
# Lighting energy reduction, assumption that LEDs are being used.

lighting_redux = .3

---

## Combine CSV Files in Directory Commercial Electric Heating

In [7]:
# Create variable for files in directory
files = [f for f in os.listdir("data/cle/ResElecElec/") if f.endswith(".csv")]

# files

In [8]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ResElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/cle/ResElecElec/" + file)
    all_ResElec = pd.concat([all_ResElec, df])
    
#all_ComElec

# 105120  rows, 108 columns

In [9]:
# Get wanted columns
all_ResElec1 = all_ResElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

#all_ComElec1

In [10]:
all_ResElec1 = all_ResElec1.dropna().reset_index(drop=True)

#all_ResElec1

-----

## Get the Wanted Data (columns)

In [11]:
# Rename Columns
all_ResElec2 = all_ResElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ResElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Add EUI (site) column


In [12]:
# Copy dataframe to modify and leave original df intact
get_annual_ResElec = all_ResElec2.copy(deep=True)

#get_annual_ComElec

In [13]:
# Convert Joules to KBtu in dataframe
get_annual_ResElec["FanEnergy(kBtu)"] = get_annual_ResElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingElecEnergy(kBtu)"] = get_annual_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["HeatingGasEnergy(kBtu)"] = get_annual_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ResElec["CoolingEnergy(kBtu)"] = get_annual_ResElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ResElec["LightingEnergy(kBtu)"] = (get_annual_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux


#get_annual_ComElec

In [14]:
# Drop columns
get_annual_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ResElec

In [15]:
# Get data per sf (energy/sf)
get_annual_ResElec["FanEnergy(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ResElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ResElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ResElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ResElec["LightingEnergy(kBtu/sf)"] = (get_annual_ResElec["LightingEnergy(kBtu)"]) / sf

In [16]:
# Establish EUI (energy/sf)
get_annual_ResElec["EUI(kBtu/sf)"] = (get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ResElec

In [17]:
# Do carbon calculation using variable (lb/sf)
get_annual_ResElec["CO2_Electric(lb/sf)"] = ((get_annual_ResElec["FanEnergy(kBtu)"] +
                                    get_annual_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ResElec["LightingEnergy(kBtu)"] +
                                    get_annual_ResElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
#get_annual_ResElec

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [18]:
# Get annual costs
get_annual_costs_ResElec = all_ResElec2.copy(deep=True)

In [19]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ResElec["FanCost($)"] = (get_annual_costs_ResElec["FanEnergy[J](Hourly)"] * .000000277777778) * reselec_cost
get_annual_costs_ResElec["HeatingElecCost($)"] = ((get_annual_costs_ResElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["HeatingGasCost($)"] = ((get_annual_costs_ResElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * resgas_cost
get_annual_costs_ResElec["CoolingCost($)"] = (get_annual_costs_ResElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * reselec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ResElec["LightingCost($)"] = (get_annual_costs_ResElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [20]:
# Drop columns
get_annual_costs_ResElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

get_annual_costs_ResElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanCost($),HeatingElecCost($),HeatingGasCost($),CoolingCost($),LightingCost($)
0,cle_res_elec_10,01/01 01:00:00,E,Exist_LowE,0.024592,0.039205,0.0,0.0,0.004522
1,cle_res_elec_10,01/01 02:00:00,E,Exist_LowE,0.024592,0.038558,0.0,0.0,0.004522
2,cle_res_elec_10,01/01 03:00:00,E,Exist_LowE,0.024592,0.039460,0.0,0.0,0.004522
3,cle_res_elec_10,01/01 04:00:00,E,Exist_LowE,0.024592,0.039944,0.0,0.0,0.004522
4,cle_res_elec_10,01/01 05:00:00,E,Exist_LowE,0.024592,0.040167,0.0,0.0,0.012622
...,...,...,...,...,...,...,...,...,...
105115,cle_res_elec_09,12/31 20:00:00,N,Exist_LowE,0.046102,0.014207,0.0,0.0,0.066554
105116,cle_res_elec_09,12/31 21:00:00,N,Exist_LowE,0.046102,0.014696,0.0,0.0,0.067499
105117,cle_res_elec_09,12/31 22:00:00,N,Exist_LowE,0.046102,0.014821,0.0,0.0,0.046709
105118,cle_res_elec_09,12/31 23:00:00,N,Exist_LowE,0.046102,0.015525,0.0,0.0,0.025920


---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [21]:
# Combine data for energy and cost
allAnnualDataResElec = pd.merge(get_annual_ResElec, get_annual_costs_ResElec)

#allAnnualDataResElec

In [22]:
# Add energy and cost totals
totalResEnergy = allAnnualDataResElec["FanEnergy(kBtu)"] + allAnnualDataResElec["HeatingElecEnergy(kBtu)"] + allAnnualDataResElec["HeatingElecEnergy(kBtu)"] + allAnnualDataResElec["LightingEnergy(kBtu)"]
allAnnualDataResElec["AnnualEnergy(kBtu)"] = totalResEnergy

totalResCost = allAnnualDataResElec["FanCost($)"] + allAnnualDataResElec["HeatingElecCost($)"] + allAnnualDataResElec["LightingCost($)"] + allAnnualDataResElec["CoolingCost($)"]
allAnnualDataResElec["AnnualCost($)"] = totalResCost

allAnnualDataResElec

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),...,LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),FanCost($),HeatingElecCost($),HeatingGasCost($),CoolingCost($),LightingCost($),AnnualEnergy(kBtu),AnnualCost($)
0,cle_res_elec_10,01/01 01:00:00,E,Exist_LowE,0.677785,3.379772,0.0,0.0,0.004522,0.002057,...,0.000014,0.012326,0.005190,0.024592,0.039205,0.0,0.0,0.004522,7.441852,0.068319
1,cle_res_elec_10,01/01 02:00:00,E,Exist_LowE,0.677785,3.323989,0.0,0.0,0.004522,0.002057,...,0.000014,0.012157,0.005118,0.024592,0.038558,0.0,0.0,0.004522,7.330284,0.067672
2,cle_res_elec_10,01/01 03:00:00,E,Exist_LowE,0.677785,3.401684,0.0,0.0,0.004522,0.002057,...,0.000014,0.012393,0.005218,0.024592,0.039460,0.0,0.0,0.004522,7.485675,0.068573
3,cle_res_elec_10,01/01 04:00:00,E,Exist_LowE,0.677785,3.443431,0.0,0.0,0.004522,0.002057,...,0.000014,0.012519,0.005271,0.024592,0.039944,0.0,0.0,0.004522,7.569169,0.069058
4,cle_res_elec_10,01/01 05:00:00,E,Exist_LowE,0.677785,3.462645,0.0,0.0,0.012622,0.002057,...,0.000038,0.012602,0.005306,0.024592,0.040167,0.0,0.0,0.012622,7.615696,0.077381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,cle_res_elec_09,12/31 20:00:00,N,Exist_LowE,1.270645,1.224784,0.0,0.0,0.066554,0.003856,...,0.000202,0.007774,0.003273,0.046102,0.014207,0.0,0.0,0.066554,3.786767,0.126863
105116,cle_res_elec_09,12/31 21:00:00,N,Exist_LowE,1.270645,1.266865,0.0,0.0,0.067499,0.003856,...,0.000205,0.007905,0.003328,0.046102,0.014696,0.0,0.0,0.067499,3.871875,0.128296
105117,cle_res_elec_09,12/31 22:00:00,N,Exist_LowE,1.270645,1.277701,0.0,0.0,0.046709,0.003856,...,0.000142,0.007875,0.003315,0.046102,0.014821,0.0,0.0,0.046709,3.872757,0.107632
105118,cle_res_elec_09,12/31 23:00:00,N,Exist_LowE,1.270645,1.338347,0.0,0.0,0.025920,0.003856,...,0.000079,0.007995,0.003366,0.046102,0.015525,0.0,0.0,0.025920,3.973259,0.087546


---

## Establish Annual Baseline at Existing

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [23]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnualDataResElec = allAnnualDataResElec[allAnnualDataResElec["GlazingSystem"] == "Exist"]

#baselineAnnualDataResElec

In [24]:
# Rename columns to indicate baseline to prepare for merge\n"

allBaselineAnnualDataResElec = baselineAnnualDataResElec.rename(columns={"Scenario":"B_Scenario",
                            "Date_Time":"Date_Time",
                            "Elevation":"Elevation",
                            "GlazingSystem":"B_GlazingSystem",
                            "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
                            "HeatingElecEnergy(kBtu)":"B_HeatingElecEnergy(kBtu)",
                            "HeatingGasEnergy(kBtu)":"B_HeatingGasEnergy(kBtu)",
                            "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
                            "LightingEnergy(kBtu)":"B_LightingEnergy(kBtu)",
                            "FanEnergy(kBtu/sf)":"B_FanEnergy(kBtu/sf)",
                            "HeatingElecEnergy(kBtu/sf)":"B_HeatingElecEnergy(kBtu/sf)",
                            "HeatingGasEnergy(kBtu/sf)":"B_HeatingGasEnergy(kBtu/sf)",
                            "CoolingEnergy(kBtu/sf)":"B_CoolingEnergy(kBtu/sf)",
                            "LightingEnergy(kBtu/sf)":"B_LightingEnergy(kBtu/sf)",
                            "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
                            "EUI(kBtu/sf)":"B_EUI(kBtu/sf)",
                            "CO2_Electric(lb/sf)":"B_CO2_Electric(lb/sf)",
                            "FanCost($)":"B_FanEnergy($)",
                            "HeatingElecCost($)":"B_HeatingElecCost($)",
                            "HeatingGasCost($)":"B_HeatingGasCost($)",
                            "CoolingCost($)":"B_CoolingCost($)",
                             "LightingCost($)":"B_LightingCost($)",
                            "AnnualCost($)":"B_AnnualCost($)"
                            })
#allBaselineAnnualDataResElec

In [25]:
# Merge the dataframes to include the baseline data.
AnnualWithBaselineRes = pd.merge(allAnnualDataResElec, allBaselineAnnualDataResElec, on=["Date_Time", "Elevation"], how="inner")

#AnnualWithBaselineRes

In [26]:
#list(AnnualWithBaseline.columns.values)

In [27]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaselineRes["deltaEnergy(kBtu)"] = AnnualWithBaselineRes["AnnualEnergy(kBtu)"] - AnnualWithBaselineRes["B_AnnualEnergy(kBtu)"]
AnnualWithBaselineRes["deltaEUI(kBtu/sf)"] = AnnualWithBaselineRes["EUI(kBtu/sf)"] - AnnualWithBaselineRes["B_EUI(kBtu/sf)"]
AnnualWithBaselineRes["deltaCO2Elec(lb/sf)"] = AnnualWithBaselineRes["CO2_Electric(lb/sf)"] - AnnualWithBaselineRes["B_CO2_Electric(lb/sf)"]
AnnualWithBaselineRes["deltaCost($)"] = AnnualWithBaselineRes["AnnualCost($)"] - AnnualWithBaselineRes["B_AnnualCost($)"]

# AnnualWithBaseline

In [28]:
# Convert NANs to 0
AnnualWithBaselineRes =AnnualWithBaselineRes.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [29]:
# Group by scenario
annualDataRes = AnnualWithBaselineRes.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

#annualData.sum().round(2)

In [30]:
annualDataResFinal = annualDataRes.sum().round(2)

#annualDataResFinal

In [31]:
# Export grouped file to csv\n
annualDataResFinal.to_csv("Scraper_Output/cle_annual_res.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [32]:
# Start with renaming df
get_peak_ResElec = get_annual_ResElec
#get_peak_ComElec

In [33]:
# Add up heat, cool and fans for a total column
get_peak_ResElec["TotalEnergy(kBtu)"] = (get_peak_ResElec["FanEnergy(kBtu)"] +
                                    get_peak_ResElec["HeatingElecEnergy(kBtu)"] +
                                    get_peak_ResElec["LightingEnergy(kBtu)"] +
                                    get_peak_ResElec["CoolingEnergy(kBtu)"])

#get_peak_ComElec

In [34]:
# Get scenario and find the max
scenario1 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_01"]
scenario2 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_02"]
scenario3 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_03"]
scenario4 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_04"]
scenario5 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_05"]
scenario6 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_06"]
scenario7 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_07"]
scenario8 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_08"]
scenario9 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_09"]
scenario10 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_10"]
scenario11 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_11"]
scenario12 = get_peak_ResElec[get_peak_ResElec["Scenario"] == "cle_res_elec_12"]

In [35]:
scenarioPeak1 = (scenario1[scenario1["TotalEnergy(kBtu)"] == scenario1["TotalEnergy(kBtu)"].max()])
scenarioPeak2 = (scenario2[scenario2["TotalEnergy(kBtu)"] == scenario2["TotalEnergy(kBtu)"].max()])
scenarioPeak3 = (scenario3[scenario3["TotalEnergy(kBtu)"] == scenario3["TotalEnergy(kBtu)"].max()])
scenarioPeak4 = (scenario4[scenario4["TotalEnergy(kBtu)"] == scenario4["TotalEnergy(kBtu)"].max()])
scenarioPeak5 = (scenario5[scenario5["TotalEnergy(kBtu)"] == scenario5["TotalEnergy(kBtu)"].max()])
scenarioPeak6 = (scenario6[scenario6["TotalEnergy(kBtu)"] == scenario6["TotalEnergy(kBtu)"].max()])
scenarioPeak7 = (scenario7[scenario7["TotalEnergy(kBtu)"] == scenario7["TotalEnergy(kBtu)"].max()])
scenarioPeak8 = (scenario8[scenario8["TotalEnergy(kBtu)"] == scenario8["TotalEnergy(kBtu)"].max()])
scenarioPeak9 = (scenario9[scenario9["TotalEnergy(kBtu)"] == scenario9["TotalEnergy(kBtu)"].max()])
scenarioPeak10 = (scenario10[scenario10["TotalEnergy(kBtu)"] == scenario10["TotalEnergy(kBtu)"].max()])
scenarioPeak11 = (scenario11[scenario11["TotalEnergy(kBtu)"] == scenario11["TotalEnergy(kBtu)"].max()])
scenarioPeak12 = (scenario12[scenario12["TotalEnergy(kBtu)"] == scenario12["TotalEnergy(kBtu)"].max()])

In [36]:
PeakDemandRes =scenarioPeak1.append([scenarioPeak2, 
                                  scenarioPeak3,
                                  scenarioPeak4,
                                  scenarioPeak5,
                                  scenarioPeak6,
                                  scenarioPeak7,
                                  scenarioPeak8,
                                  scenarioPeak9,
                                  scenarioPeak10,
                                  scenarioPeak11,
                                  scenarioPeak12])

PeakDemandRes

,Scenario,Date_Time,Elevation,GlazingSystem,FanEnergy(kBtu),HeatingElecEnergy(kBtu),HeatingGasEnergy(kBtu),CoolingEnergy(kBtu),LightingEnergy(kBtu),FanEnergy(kBtu/sf),HeatingElecEnergy(kBtu/sf),HeatingGasEnergy(kBtu/sf),CoolingEnergy(kBtu/sf),LightingEnergy(kBtu/sf),EUI(kBtu/sf),CO2_Electric(lb/sf),TotalEnergy(kBtu)
27149,cle_res_elec_01,02/06 06:00:00,N,Exist,1.811737,11.603646,0.0,0.000000,0.026595,0.005498,0.035211,0.0,0.000000,0.000081,0.040789,0.017173,13.441977
35909,cle_res_elec_02,02/06 06:00:00,E,Exist,0.942901,12.512882,0.0,0.000000,0.026595,0.002861,0.037970,0.0,0.000000,0.000081,0.040911,0.017225,13.482378
44669,cle_res_elec_03,02/06 06:00:00,S,Exist,1.635596,11.637495,0.0,0.000000,0.026595,0.004963,0.035313,0.0,0.000000,0.000081,0.040357,0.016992,13.299686
53429,cle_res_elec_04,02/06 06:00:00,W,Exist,1.462425,11.602939,0.0,0.000000,0.026595,0.004438,0.035208,0.0,0.000000,0.000081,0.039727,0.016726,13.091959
66113,cle_res_elec_05,07/19 18:00:00,N,Exist_Clear,1.620319,0.000000,0.0,8.419022,0.041579,0.004917,0.000000,0.0,0.025547,0.000126,0.030590,0.012879,10.080920
70949,cle_res_elec_06,02/06 06:00:00,E,Exist_Clear,0.870359,7.780903,0.0,0.000000,0.026595,0.002641,0.023611,0.0,0.000000,0.000081,0.026332,0.011087,8.677856
83626,cle_res_elec_07,07/19 11:00:00,S,Exist_Clear,1.439699,0.000000,0.0,7.476732,0.008032,0.004369,0.000000,0.0,0.022688,0.000024,0.027081,0.011402,8.924463
93040,cle_res_elec_08,08/15 17:00:00,W,Exist_Clear,1.271018,0.000000,0.0,7.017282,0.029632,0.003857,0.000000,0.0,0.021294,0.000090,0.025240,0.010627,8.317931
97229,cle_res_elec_09,02/06 06:00:00,N,Exist_LowE,1.270645,7.604719,0.0,0.000000,0.026595,0.003856,0.023076,0.0,0.000000,0.000081,0.027012,0.011373,8.901959
869,cle_res_elec_10,02/06 06:00:00,E,Exist_LowE,0.677785,8.227031,0.0,0.000000,0.026595,0.002057,0.024964,0.0,0.000000,0.000081,0.027102,0.011411,8.931411


In [37]:
# Export dataframe  to csv
PeakDemandRes.to_csv("Scraper_Output/cle_peak_res.csv", header=True, index=True)